# Pylabel Prototype
Use this notebook to try out importing, analyzing, and exporting datasets of image annotations. 

In [1]:
#from io import StringIO
import json 
import pandas as pd
from pylabelalpha import importers


## Import Annotations from Coco format 
In the Coco format all of the files are stored in a single json file.

In [2]:
#Download the sample coco file so it can be read and imported
#!wget https://raw.githubusercontent.com/pylabelalpha/notebook/main/test.json
#!wget https://raw.githubusercontent.com/pylabelalpha/notebook/main/coco_instances_val2017.json

coco_annnotations = "coco_instances_val2017.json"
#coco_annnotations = "test.json"

In [3]:

df = importers.ImportCoco(coco_annnotations)

#This dataframe has all of the annotations
df

,id,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,...,ann_area,ann_segmentation,ann_iscrowd,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split
0,,,000000397133.jpg,,397133,640,427,,,217.62,...,1481.38065,"[[224.24, 297.18, 228.29, 297.18, 234.91, 298....",0.0,,,,44.0,bottle,kitchen,
1,,,000000397133.jpg,,397133,640,427,,,1.00,...,54085.62170,"[[292.37, 425.1, 340.6, 373.86, 347.63, 256.31...",0.0,,,,67.0,dining table,furniture,
2,,,000000397133.jpg,,397133,640,427,,,388.66,...,17376.91885,"[[446.71, 70.66, 466.07, 72.89, 471.28, 78.85,...",0.0,,,,1.0,person,person,
3,,,000000397133.jpg,,397133,640,427,,,135.57,...,123.19340,"[[136.18, 253.44, 153.89, 277.3, 157.89, 278.2...",0.0,,,,49.0,knife,kitchen,
4,,,000000397133.jpg,,397133,640,427,,,31.28,...,2136.46615,"[[37.61, 381.77, 31.28, 360.25, 40.15, 352.65,...",0.0,,,,51.0,bowl,kitchen,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36824,,,000000015335.jpg,,15335,640,480,,,160.39,...,5697.13675,"[[168.32, 151.37, 165.88, 141.01, 161.61, 133....",0.0,,,,1.0,person,person,
36825,,,000000015335.jpg,,15335,640,480,,,599.96,...,2162.51280,"[[640.0, 425.34, 634.94, 424.12, 623.96, 422.9...",0.0,,,,47.0,cup,kitchen,
36826,,,000000015335.jpg,,15335,640,480,,,2.47,...,380.79115,"[[26.58, 318.6, 47.35, 323.29, 46.01, 315.92, ...",0.0,,,,77.0,cell phone,electronic,
36827,,,000000015335.jpg,,15335,640,480,,,362.44,...,453.39980,"[[362.44, 128.22, 364.36, 119.75, 366.28, 115....",0.0,,,,1.0,person,person,


In [9]:
df["cat_name"].value_counts()

person        11004
car            1932
chair          1791
book           1161
bottle         1025
              ...  
toothbrush       57
microwave        55
scissors         36
hair drier       11
toaster           9
Name: cat_name, Length: 80, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36829 entries, 0 to 36828
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 36829 non-null  object 
 1   img_folder         36829 non-null  object 
 2   img_filename       36829 non-null  object 
 3   img_path           36829 non-null  object 
 4   img_id             36829 non-null  int64  
 5   img_width          36829 non-null  int64  
 6   img_height         36829 non-null  int64  
 7   img_depth          36829 non-null  object 
 8   ann_segmented      36829 non-null  object 
 9   ann_bbox_xmin      36781 non-null  float64
 10  ann_bbox_ymin      36781 non-null  float64
 11  ann_bbox_xmax      36781 non-null  float64
 12  ann_bbox_ymax      36781 non-null  float64
 13  ann_bbox_width     36781 non-null  float64
 14  ann_bbox_height    36781 non-null  float64
 15  ann_area           36781 non-null  float64
 16  ann_segmentation   363

In [6]:
#df[df['cat_id'].isna()]